# Tendering Analysis System - Lab Notebook

Este notebook demuestra la funcionalidad de los diferentes agentes.

## Environment Setup

In [1]:
from pathlib import Path
import sys
import os
from dotenv import load_dotenv

# Try to import and load dotenv
try:
    load_dotenv()
    print("🔧 Environment variables loaded from .env file")
    print(f"📄 OpenAI API Key configured: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")
except ImportError:
    print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")
    print("📄 Using system environment variables only")

🔧 Environment variables loaded from .env file
📄 OpenAI API Key configured: Yes


In [2]:
# Get current working directory and navigate to the backend directory
current_dir = Path(os.getcwd())
# If we're in the backend folder, go to parent (tendering_app), otherwise assume we're already there
backend_dir = current_dir if current_dir.name == "tendering_app" else current_dir.parent
sys.path.append(str(backend_dir))

In [4]:
from utils.agents.document_classification import DocumentClassificationAgent

"""Ejemplo básico de uso del agente de clasificación"""

print("🔍 DocumentClassificationAgent - Ejemplo de Uso")
print("=" * 50)

# 1. Crear instancia del agente
document_path = backend_dir / "documents" / "EJEMPLO DE CONTRATO - RETO 1.pdf"

classiffication_agent = DocumentClassificationAgent(
    document_path=document_path,
    vector_db_path=backend_dir / "classification_db",
    collection_name="EjemploContrato"
)

print(f"📄 Documento a procesar: {document_path.name}")

# 2. Procesar documento completo
print("\n⚙️ Procesando documento...")

try:
    # Usar provider automático (OLLAMA si está disponible, sino OpenAI)
    report = classiffication_agent.process_document(
        provider="auto",  # o "ollama" / "openai" específicamente
        force_rebuild=True  # Reconstruir embeddings
    )
    
    if "error" in report:
        print(f"❌ Error: {report['error']}")
    
    # 3. Mostrar estructura del documento
    print("\n📋 Estructura del Documento:")
    print("-" * 30)
    
    for section_name, section_info in report['sections'].items():
        confidence = report['confidence_scores'].get(section_name, 0)
        print(f"📄 {section_name}")
        print(f"   └─ Fragmentos: {section_info['document_count']}")
        print(f"   └─ Caracteres: {section_info['total_characters']:,}")
        print(f"   └─ Confianza: {confidence:.1f}%")
        
        if section_info.get('content_preview'):
            preview = section_info['content_preview'][:1000]
            print(f"   └─ Vista previa: {preview}...")
        print()
    
    # 4. Mostrar requisitos clave
    if report['key_requirements']:
        print("🔍 Requisitos Clave Encontrados:")
        print("-" * 30)
        
        for section, requirements in report['key_requirements'].items():
            if requirements:
                print(f"\n📋 {section}:")
                for i, req in enumerate(requirements[:3], 1):
                    print(f"   {i}. {req[:100]}...")
    
    # 5. Ejemplo de búsqueda semántica
    print("\n🔍 Ejemplo de Búsqueda Semántica:")
    print("-" * 30)
    
    queries = [
        "requisitos técnicos",
        "condiciones económicas",
        "garantías necesarias"
    ]
    
    for query in queries:
        print(f"\n🔎 Búsqueda: '{query}'")
        results = classiffication_agent.semantic_search(query, top_k=2)
        
        for i, (doc, score) in enumerate(results, 1):
            section = doc.metadata.get('section', 'GENERAL')
            preview = doc.page_content[:80].replace('\n', ' ')
            print(f"   {i}. [Score: {score:.3f}] [{section}]")
            print(f"      {preview}...")
    
    # 6. Estadísticas finales
    print(f"\n📊 Resumen:")
    print(f"   Total secciones: {report['document_info']['total_sections']}")
    print(f"   Total fragmentos: {report['document_info']['total_fragments']}")
    print(f"   Documento fuente: {report['document_info']['source']}")
    
    print("\n✅ Procesamiento completado exitosamente!")
    
except Exception as e:
    print(f"❌ Error durante el procesamiento: {e}")
    import traceback
    traceback.print_exc()

🔍 DocumentClassificationAgent - Ejemplo de Uso
📄 Documento a procesar: EJEMPLO DE CONTRATO - RETO 1.pdf

⚙️ Procesando documento...

📋 Estructura del Documento:
------------------------------
📄 CONVOCATORIA
   └─ Fragmentos: 10
   └─ Caracteres: 13,058
   └─ Confianza: 0.0%
   └─ Vista previa: ACTA N.º 001-2025-PG
Fecha: [día/mes/2025]
Adjudicado a: EDIFIKA S.A.
Monto: USD 20,000,000.00
Plazo: 12 meses
Firmas: Prefectura del Guayas y EDIFIKA S.A.......

📄 OBJETO
   └─ Fragmentos: 10
   └─ Caracteres: 14,470
   └─ Confianza: 20.0%
   └─ Vista previa: PRIMERA – OBJETO DEL CONTRATO
La PREFECTURA DEL GUAYAS encarga a EDIFIKA S.A. RUC: 0992881364001, la ejecución de la
obra denominada “Ampliación de la Vía Samborondón – Asfaltado completo de 10 carril......

📄 CONDICIONES_GENERALES
   └─ Fragmentos: 10
   └─ Caracteres: 13,625
   └─ Confianza: 0.0%
   └─ Vista previa: PRIMERA – OBJETO DEL CONTRATO
La PREFECTURA DEL GUAYAS encarga a EDIFIKA S.A. RUC: 0992881364001, la ejecución de la
obra de